The purpose of this script is to remove all variants falling in regions of the genome we expect to be uninformative. This whole project is . about evolutionary conservation and malinois : an algorithm that predicts cis-regulatory activity. Any relationships between these two variables would be obscured by the presence of genetic sequnces which could affect conservation for some reason other than their cis-regulatory activity. For this reason, I will exclude exons and splice-sites (File from Dr. Rong : exons of cannonical transcripts plus 20 bases into the splice acceptor and 6 bases into the splice donor  (numbers are from MaxEntScan motifs) ).

Initalize the spark session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import json

spark = SparkSession.builder.appName("filter").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/16 16:54:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/16 16:54:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Load the data

In [15]:
#loading the extended-exon bed file
#note we are discarding all information other than chromosome, start, & stop
#!!note "autosome" : we are wholey exclusing the sex chromosomes!!
exons = spark.read.csv("/home/mcn26/varef/scripts/noon_data/gencode.v44.protein.coding.exons.splice.autosomes.bed", header=False, inferSchema=True,sep="\t").select(
    F.col('_c0').alias('chromosome'),
    F.col('_c1').alias('start'),
    F.col('_c2').alias('end')
)

#repartition the data: we want it organized by chromosome
large_table_df = large_table_df.repartition("chromosome")

In [24]:
#as noted above: we are only examining autosomes. 


['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']


In [16]:
exons.take(3)

[Row(chromosome='chr1', start=65418, end=65433),
 Row(chromosome='chr1', start=65499, end=65579),
 Row(chromosome='chr1', start=69036, end=71585)]

Our expanded exons are in a BED file.
BED files are 0-based

Our incoming data was taken from a VCF originally
VCF files are 1-based


In [14]:
x="""
Chr1        T   A   C   G   T
          | | | | | | | | | |
1 based   | 1 | 2 | 3 | 4 | 5
0 based   0   1   2   3   4
"""